In [0]:
pip install soundfile

In [0]:
!pip install -q torch==1.3.1 torchvision

In [0]:
import seaborn
import pandas as pd
from google.colab import files
import numpy as np
from tqdm import tqdm
import soundfile as sf
import librosa
import os
import dask as dd
import sklearn 
import torch

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
pip install python_speech_features

In [0]:
pip install pydub

In [0]:
import numpy as np
import scipy.io.wavfile as wave
import python_speech_features as psf
from pydub import AudioSegment

In [0]:
def getSignal(path):
    #open wav file
    (rate,sig) = wave.read(path)
    sig = sig[:22050] #Was 22050
    #print(len(sig))
    return sig

#Obtain a list of all the numpy arrays of the test and training signals
test_files = os.listdir('/content/drive/Shared drives/301FP/LPF_Test_Files_Shortened')
training_files = os.listdir('/content/drive/Shared drives/301FP/unzipped/LPF_train/filtered_files')
print (len(test_files))
print (len(training_files))
test_sigs = [getSignal('/content/drive/Shared drives/301FP/LPF_Test_Files_Shortened/' + x) for x in test_files]
print('Finished the test files')
train_sigs = [getSignal('/content/drive/Shared drives/301FP/unzipped/LPF_train/filtered_files/' + y) for y in training_files]
print('Finished the training files')

541
2757
Finished the test files
Finished the training files


In [0]:
import csv

#Get the training data, labels, and test signals for KNN
training_data = pd.read_csv('/content/drive/Shared drives/301FP/Data_Folder/train.csv')
#print(training_data) #To verify the training data is the same length as what I downloaded from our drive
train_sigs = list(train_sigs)
test_sigs = list(test_sigs)
files = training_data['fname'].tolist()
labels = training_data['label'].tolist()

#Fine tune this base implementation of KNN
def KNN(num_of_neighbors, norm, train_data, train_data2, file_names, test_data, labelnames):
  """
  Inputs:
  num_of_neighbors: An integer representing how many neighbors to consider for classification
  norm: An integer representing the norm to use for calculating distance
  train_data: A list containing all of the training files
  train_data2: A list containing all of the training file names, corresponding to train_data
  train_data_names: A list of all the names of the .wav training files
  test_data: A list containing all of the test files
  labelnames: A list containing the labels of the training data

  Returns: A list containing the test data's corresponding labels
  """
  num_of_trains = len(train_data)
  num_of_tests = len(test_data)
  our_test = [0 for i in range(len(test_data))]
  for i in range(num_of_tests):
    norms = [0 for a in range(num_of_trains)]
    smallest_indices = [0 for a in range(num_of_neighbors)]
    for j in range(num_of_trains):
      norms[j] = (sum([(test_data[j][b]-train_data[i][b])**norm for b in range(0,22050)]))**(1.0/norm)
    for k in range(num_of_neighbors):
      minimum = min(norms)
      idx = norms.index(minimum)
      norms[idx] = float('inf')
      best_fit = train_data2[idx]
      idx2 = file_names.index(best_fit)
      smallest_indices[k] = labelnames(idx2)
    mode = max(set(smallest_indices), key = smallest_indices.count)
    our_test[i] = mode
  return our_test

(test, label) = KNN(9, 2, train_sigs, training_files, files, test_sigs, labels)